In [5]:
from __future__ import print_function, division
import tensorflow as tf
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np

class BGAN():
    """Reference: https://wiseodd.github.io/techblog/2017/03/07/boundary-seeking-gan/"""
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.boundary_loss, optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def boundary_loss(self, y_true, y_pred):
        """
        Boundary seeking loss.
        Reference: https://wiseodd.github.io/techblog/2017/03/07/boundary-seeking-gan/
        """
        return 0.5 * K.mean((K.log(y_pred) - K.log(1 - y_pred))**2)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("C:/Users/kaka/PycharmProjects/pythonProject/AOI/ICPR2023/Mnist/images/mnist_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    bgan = BGAN()
    bgan.train(epochs=30000, batch_size=32, sample_interval=200)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_7 (Dense)             (None, 512)               401920    
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_8 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-tr

92 [D loss: 0.005570, acc.: 100.00%] [G loss: 13.378700]
93 [D loss: 0.005936, acc.: 100.00%] [G loss: 12.928196]
94 [D loss: 0.007749, acc.: 100.00%] [G loss: 14.645397]
95 [D loss: 0.007438, acc.: 100.00%] [G loss: 13.905901]
96 [D loss: 0.009212, acc.: 100.00%] [G loss: 12.991947]
97 [D loss: 0.003183, acc.: 100.00%] [G loss: 12.646380]
98 [D loss: 0.006120, acc.: 100.00%] [G loss: 12.756564]
99 [D loss: 0.008150, acc.: 100.00%] [G loss: 13.246941]
100 [D loss: 0.011846, acc.: 100.00%] [G loss: 13.701167]
101 [D loss: 0.016751, acc.: 100.00%] [G loss: 14.228358]
102 [D loss: 0.005660, acc.: 100.00%] [G loss: 14.649526]
103 [D loss: 0.014204, acc.: 100.00%] [G loss: 13.937532]
104 [D loss: 0.010941, acc.: 100.00%] [G loss: 15.657360]
105 [D loss: 0.005916, acc.: 100.00%] [G loss: 14.196348]
106 [D loss: 0.007909, acc.: 100.00%] [G loss: 14.353509]
107 [D loss: 0.008189, acc.: 100.00%] [G loss: 15.078897]
108 [D loss: 0.006819, acc.: 100.00%] [G loss: 13.270102]
109 [D loss: 0.010003,

238 [D loss: 0.262319, acc.: 89.06%] [G loss: 6.129285]
239 [D loss: 0.695884, acc.: 62.50%] [G loss: 2.686831]
240 [D loss: 0.511313, acc.: 78.12%] [G loss: 4.781765]
241 [D loss: 0.614833, acc.: 67.19%] [G loss: 1.425775]
242 [D loss: 0.254244, acc.: 90.62%] [G loss: 5.561074]
243 [D loss: 0.989883, acc.: 46.88%] [G loss: 0.638388]
244 [D loss: 0.232477, acc.: 93.75%] [G loss: 4.357811]
245 [D loss: 0.601137, acc.: 67.19%] [G loss: 1.796991]
246 [D loss: 0.348612, acc.: 85.94%] [G loss: 6.177014]
247 [D loss: 0.703033, acc.: 53.12%] [G loss: 0.759774]
248 [D loss: 0.229360, acc.: 87.50%] [G loss: 5.074234]
249 [D loss: 0.880902, acc.: 53.12%] [G loss: 0.732860]
250 [D loss: 0.383302, acc.: 71.88%] [G loss: 6.076547]
251 [D loss: 0.838041, acc.: 46.88%] [G loss: 1.246724]
252 [D loss: 0.271983, acc.: 93.75%] [G loss: 4.002960]
253 [D loss: 0.612881, acc.: 64.06%] [G loss: 0.653960]
254 [D loss: 0.353912, acc.: 82.81%] [G loss: 3.808091]
255 [D loss: 0.578517, acc.: 71.88%] [G loss: 2.

387 [D loss: 0.625034, acc.: 64.06%] [G loss: 0.403996]
388 [D loss: 0.556615, acc.: 76.56%] [G loss: 0.258258]
389 [D loss: 0.550584, acc.: 73.44%] [G loss: 0.326848]
390 [D loss: 0.591388, acc.: 70.31%] [G loss: 0.237743]
391 [D loss: 0.651308, acc.: 54.69%] [G loss: 0.176952]
392 [D loss: 0.571507, acc.: 75.00%] [G loss: 0.220793]
393 [D loss: 0.547494, acc.: 75.00%] [G loss: 0.440530]
394 [D loss: 0.661590, acc.: 54.69%] [G loss: 0.317188]
395 [D loss: 0.603213, acc.: 67.19%] [G loss: 0.390139]
396 [D loss: 0.613264, acc.: 65.62%] [G loss: 0.231228]
397 [D loss: 0.670496, acc.: 46.88%] [G loss: 0.184566]
398 [D loss: 0.552257, acc.: 76.56%] [G loss: 0.424143]
399 [D loss: 0.566338, acc.: 71.88%] [G loss: 0.298286]
400 [D loss: 0.647325, acc.: 56.25%] [G loss: 0.251444]
401 [D loss: 0.579472, acc.: 62.50%] [G loss: 0.322901]
402 [D loss: 0.593033, acc.: 75.00%] [G loss: 0.285257]
403 [D loss: 0.596295, acc.: 70.31%] [G loss: 0.235215]
404 [D loss: 0.628323, acc.: 59.38%] [G loss: 0.

535 [D loss: 0.571406, acc.: 73.44%] [G loss: 0.214440]
536 [D loss: 0.601879, acc.: 68.75%] [G loss: 0.150952]
537 [D loss: 0.560054, acc.: 78.12%] [G loss: 0.232446]
538 [D loss: 0.570489, acc.: 79.69%] [G loss: 0.106657]
539 [D loss: 0.630011, acc.: 59.38%] [G loss: 0.152566]
540 [D loss: 0.585631, acc.: 62.50%] [G loss: 0.136937]
541 [D loss: 0.599907, acc.: 68.75%] [G loss: 0.118038]
542 [D loss: 0.560126, acc.: 75.00%] [G loss: 0.182924]
543 [D loss: 0.536390, acc.: 79.69%] [G loss: 0.130693]
544 [D loss: 0.547073, acc.: 79.69%] [G loss: 0.171845]
545 [D loss: 0.581626, acc.: 68.75%] [G loss: 0.161151]
546 [D loss: 0.547439, acc.: 71.88%] [G loss: 0.194032]
547 [D loss: 0.578444, acc.: 73.44%] [G loss: 0.174961]
548 [D loss: 0.640610, acc.: 67.19%] [G loss: 0.122944]
549 [D loss: 0.590321, acc.: 64.06%] [G loss: 0.190436]
550 [D loss: 0.538137, acc.: 76.56%] [G loss: 0.187339]
551 [D loss: 0.569632, acc.: 70.31%] [G loss: 0.190497]
552 [D loss: 0.575475, acc.: 78.12%] [G loss: 0.

...

29373 [D loss: 0.662430, acc.: 65.62%] [G loss: 0.884052]
29374 [D loss: 0.561859, acc.: 68.75%] [G loss: 0.624285]
29375 [D loss: 0.489105, acc.: 79.69%] [G loss: 0.626052]
29376 [D loss: 0.494439, acc.: 78.12%] [G loss: 0.526144]
29377 [D loss: 0.542385, acc.: 70.31%] [G loss: 0.720106]
29378 [D loss: 0.517042, acc.: 71.88%] [G loss: 0.782266]
29379 [D loss: 0.573746, acc.: 65.62%] [G loss: 0.831746]
29380 [D loss: 0.538023, acc.: 71.88%] [G loss: 1.134508]
29381 [D loss: 0.503590, acc.: 73.44%] [G loss: 1.058486]
29382 [D loss: 0.559960, acc.: 71.88%] [G loss: 0.712697]
29383 [D loss: 0.448870, acc.: 79.69%] [G loss: 0.733340]
29384 [D loss: 0.575669, acc.: 68.75%] [G loss: 0.687028]
29385 [D loss: 0.540020, acc.: 73.44%] [G loss: 0.810873]
29386 [D loss: 0.543693, acc.: 68.75%] [G loss: 0.736897]
29387 [D loss: 0.549429, acc.: 67.19%] [G loss: 0.914402]
29388 [D loss: 0.500339, acc.: 68.75%] [G loss: 0.700327]
29389 [D loss: 0.630017, acc.: 56.25%] [G loss: 0.646852]
29390 [D loss:

29516 [D loss: 0.517062, acc.: 73.44%] [G loss: 0.766709]
29517 [D loss: 0.554330, acc.: 70.31%] [G loss: 0.768344]
29518 [D loss: 0.494940, acc.: 71.88%] [G loss: 1.064631]
29519 [D loss: 0.436127, acc.: 78.12%] [G loss: 1.106011]
29520 [D loss: 0.512571, acc.: 73.44%] [G loss: 0.880223]
29521 [D loss: 0.600531, acc.: 65.62%] [G loss: 0.836358]
29522 [D loss: 0.514831, acc.: 73.44%] [G loss: 0.732038]
29523 [D loss: 0.509600, acc.: 73.44%] [G loss: 0.765240]
29524 [D loss: 0.534879, acc.: 70.31%] [G loss: 0.718465]
29525 [D loss: 0.472513, acc.: 75.00%] [G loss: 0.897311]
29526 [D loss: 0.441280, acc.: 78.12%] [G loss: 0.918629]
29527 [D loss: 0.442463, acc.: 76.56%] [G loss: 0.893148]
29528 [D loss: 0.586567, acc.: 70.31%] [G loss: 1.086424]
29529 [D loss: 0.483867, acc.: 76.56%] [G loss: 0.872926]
29530 [D loss: 0.544219, acc.: 70.31%] [G loss: 0.832569]
29531 [D loss: 0.529337, acc.: 73.44%] [G loss: 0.736446]
29532 [D loss: 0.427486, acc.: 78.12%] [G loss: 0.829304]
29533 [D loss:

29660 [D loss: 0.491866, acc.: 76.56%] [G loss: 0.809200]
29661 [D loss: 0.475975, acc.: 79.69%] [G loss: 0.954704]
29662 [D loss: 0.481569, acc.: 75.00%] [G loss: 0.750172]
29663 [D loss: 0.400987, acc.: 85.94%] [G loss: 0.908622]
29664 [D loss: 0.638855, acc.: 60.94%] [G loss: 0.783289]
29665 [D loss: 0.490449, acc.: 73.44%] [G loss: 0.995424]
29666 [D loss: 0.453411, acc.: 76.56%] [G loss: 1.015668]
29667 [D loss: 0.639797, acc.: 64.06%] [G loss: 1.009174]
29668 [D loss: 0.544736, acc.: 68.75%] [G loss: 0.844769]
29669 [D loss: 0.476531, acc.: 71.88%] [G loss: 0.755063]
29670 [D loss: 0.478714, acc.: 76.56%] [G loss: 0.956450]
29671 [D loss: 0.574959, acc.: 70.31%] [G loss: 0.694018]
29672 [D loss: 0.485151, acc.: 78.12%] [G loss: 0.990648]
29673 [D loss: 0.523605, acc.: 75.00%] [G loss: 1.053379]
29674 [D loss: 0.542533, acc.: 67.19%] [G loss: 0.900575]
29675 [D loss: 0.511375, acc.: 70.31%] [G loss: 0.972764]
29676 [D loss: 0.597238, acc.: 68.75%] [G loss: 1.248121]
29677 [D loss:

29805 [D loss: 0.524746, acc.: 73.44%] [G loss: 0.701349]
29806 [D loss: 0.529176, acc.: 67.19%] [G loss: 0.901014]
29807 [D loss: 0.638822, acc.: 59.38%] [G loss: 0.893174]
29808 [D loss: 0.476360, acc.: 71.88%] [G loss: 0.746395]
29809 [D loss: 0.371908, acc.: 84.38%] [G loss: 1.125054]
29810 [D loss: 0.602091, acc.: 60.94%] [G loss: 0.717440]
29811 [D loss: 0.436933, acc.: 76.56%] [G loss: 1.057744]
29812 [D loss: 0.539034, acc.: 71.88%] [G loss: 0.843627]
29813 [D loss: 0.655266, acc.: 62.50%] [G loss: 1.030617]
29814 [D loss: 0.544366, acc.: 67.19%] [G loss: 1.086096]
29815 [D loss: 0.499026, acc.: 71.88%] [G loss: 1.102960]
29816 [D loss: 0.521476, acc.: 73.44%] [G loss: 0.612280]
29817 [D loss: 0.469944, acc.: 79.69%] [G loss: 0.792194]
29818 [D loss: 0.551245, acc.: 59.38%] [G loss: 0.845111]
29819 [D loss: 0.462784, acc.: 76.56%] [G loss: 1.499774]
29820 [D loss: 0.548760, acc.: 70.31%] [G loss: 0.881396]
29821 [D loss: 0.511505, acc.: 71.88%] [G loss: 1.153360]
29822 [D loss:

29950 [D loss: 0.584703, acc.: 70.31%] [G loss: 0.932554]
29951 [D loss: 0.674044, acc.: 65.62%] [G loss: 0.680187]
29952 [D loss: 0.540700, acc.: 65.62%] [G loss: 0.731109]
29953 [D loss: 0.568870, acc.: 62.50%] [G loss: 0.877830]
29954 [D loss: 0.425451, acc.: 78.12%] [G loss: 0.883958]
29955 [D loss: 0.486455, acc.: 70.31%] [G loss: 0.854764]
29956 [D loss: 0.456775, acc.: 76.56%] [G loss: 0.919232]
29957 [D loss: 0.534960, acc.: 75.00%] [G loss: 0.838944]
29958 [D loss: 0.502841, acc.: 70.31%] [G loss: 1.017154]
29959 [D loss: 0.605746, acc.: 67.19%] [G loss: 0.760795]
29960 [D loss: 0.472344, acc.: 76.56%] [G loss: 0.562306]
29961 [D loss: 0.497568, acc.: 68.75%] [G loss: 0.791425]
29962 [D loss: 0.585192, acc.: 64.06%] [G loss: 0.631130]
29963 [D loss: 0.542817, acc.: 65.62%] [G loss: 0.613867]
29964 [D loss: 0.412899, acc.: 78.12%] [G loss: 1.066817]
29965 [D loss: 0.430365, acc.: 82.81%] [G loss: 1.023827]
29966 [D loss: 0.508505, acc.: 75.00%] [G loss: 0.821117]
29967 [D loss: